# Introduction

In this notebook we demonstrate the use of **LDA (Latent Dirichlet Allocation)** generative statistical model for Information Retrieval technique to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each test case content as an entire document that must be returned to the query made

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from dit.divergences import jensen_shannon_divergence

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from utils import plots
from utils import firefox_dataset as fd
from utils import tokenizers as tok
from utils import aux_functions
from utils import model_evaluator as m_eval

from models.lda import LDA
from models.model_hyperps import LDA_Model_Hyperp

from IPython.display import display

import warnings; warnings.simplefilter('ignore')

## Load Dataset

In [2]:
test_cases_df = fd.read_testcases_df()
bug_reports_df = fd.read_bugreports_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.read_trace_df()
orc.set_index('tc_name', inplace=True, drop=True)

print(bug_reports_df.shape)
print(test_cases_df.shape)
print(orc.shape)

(35314, 12)
(207, 10)
(207, 35314)


## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

### Analysis with Default Values of LDA Model

In [6]:
model = LDA()
model.recover_links(corpus, query, use_cases_names, bug_reports_names)
evaluator = m_eval.ModelEvaluator(orc.oracle, model)
evaluator.evaluate_model(verbose=True)

{'Measures': {'Mean FScore of LDA': 0.0,
              'Mean Precision of LDA': 0.0,
              'Mean Recall of LDA': 0.0},
 'Setup': [{'Name': 'LDA'},
           {'Similarity Measure and Minimum Threshold': ('jsd', 0.3)},
           {'Top Value': 3},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': None,
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 10,
                          'n_jobs': -1,
                          'n_topics': None,
                          'perp_tol': 0.1,
                          'random_state': None,
                          'topic_word_prior': None,
                          'to

## Running LDA Model with Different Types of Oracles

### Define **run_lda_model()** Function

In [6]:
def run_lda_model(selected_tcs, selected_brs):
    tcs_df = test_cases_df[test_cases_df.tc_name.isin(selected_tcs)]
    brs_df = bug_reports_df[bug_reports_df.br_name.isin(selected_brs)]

    corpus_subset = tcs_df.tc_desc
    query_subset = brs_df.br_desc
    testcases_names_subset = tcs_df.tc_name
    bug_reports_names_subset = brs_df.br_name
    orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

    print('TestCases Subset Shape: {}'.format(tcs_df.shape))
    print('BugReports Subset Shape: {}'.format(brs_df.shape))
    print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

    lda_hyperp = {
        LDA_Model_Hyperp.TOP.value : 5,
        LDA_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : ('cosine',.75),
        LDA_Model_Hyperp.LDA_MODEL_N_COMPONENTS.value: 50,
        LDA_Model_Hyperp.LDA_MODEL_RANDOM_STATE.value : 2,
        LDA_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: (1,1),
        LDA_Model_Hyperp.VECTORIZER.value : TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True),
        LDA_Model_Hyperp.VECTORIZER_TOKENIZER.value : tok.PorterStemmerBased_Tokenizer() 
    }

    lda_model = LDA(**lda_hyperp)
    lda_model.set_name('LDA_Model_0')
    lda_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)

    print("\nModel Evaluation -------------------------------------------")
    evaluator = m_eval.ModelEvaluator(orc_subset_df, lda_model)
    evaluator.evaluate_model(verbose=True)

    print("\n\nTraceLinks Matrix --------------------------------------")
    display(aux_functions.highlight_df(lda_model.get_trace_links_df()))
    
    print("\n\nOracle -----------------------------------------")
    display(aux_functions.highlight_df(orc_subset_df))

### Oracle with Strong Links Only

In [ ]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in []]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in []]

run_lda_model(selected_tcs, selected_brs)

### Oracle with Weak Links Only

In [ ]:
selected_tcs = []
selected_brs = []

run_lda_model(selected_tcs, selected_brs)

### Oracle with Mixed Links (Strong and Weak)

In [ ]:
selected_tcs = []
selected_brs = []

run_lda_model(selected_tcs, selected_brs)

### General Test

In [7]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

selected_bugreports = bugreports_subset_df.br_name
run_lda_model(selected_testcases, selected_bugreports)


TestCases Subset Shape: (6, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (6, 15)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LDA_Model_0': 0.15333333333333332,
              'Mean Precision of LDA_Model_0': 0.26666666666666666,
              'Mean Recall of LDA_Model_0': 0.1111111111111111},
 'Setup': [{'Name': 'LDA_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'LDA Model': {'batch_size': 128,
                          'doc_topic_prior': None,
                          'evaluate_every': -1,
                          'learning_decay': 0.7,
                          'learning_method': 'batch',
                          'learning_offset': 10.0,
                          'max_doc_update_iter': 100,
                          'max_iter': 10,
                          'mean_change_tol': 0.001,
                          'n_components': 50,
             

br_name,BR_1268934_SRC,BR_1282551_SRC,BR_1291175_SRC,BR_1299787_SRC,BR_1418983_SRC,BR_1432520_SRC,BR_1436749_SRC,BR_1443632_SRC,BR_1443754_SRC,BR_1450216_SRC,BR_1461828_SRC,BR_1463274_SRC,BR_1463735_SRC,BR_1497738_SRC,BR_1513270_SRC
tc_name,,,,,,,,,,,,,,,
TC_13_TRG,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1
TC_14_TRG,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1
TC_15_TRG,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0
TC_16_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
TC_17_TRG,1,0,0,1,1,1,1,0,1,0,1,0,0,0,1
TC_18_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1




Oracle -----------------------------------------


,BR_1268934_SRC,BR_1282551_SRC,BR_1291175_SRC,BR_1299787_SRC,BR_1418983_SRC,BR_1432520_SRC,BR_1436749_SRC,BR_1443632_SRC,BR_1443754_SRC,BR_1450216_SRC,BR_1461828_SRC,BR_1463274_SRC,BR_1463735_SRC,BR_1497738_SRC,BR_1513270_SRC
tc_name,,,,,,,,,,,,,,,
TC_13_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_14_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_17_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_18_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
